Fluxo de Projeto em sistemas embarcados - 2025.1
#### Prof. Josenalde Barbosa de Oliveira - PPgMECA@UFRN

### Atividade 3.1: desenvolver código com paradigma paralelo para estimar o número PI

Embora existam outras formas, para nosso objetivo o número PI pode ser aproximado pela série de N termos:

$\pi = 4 \left ( \frac{1}{1} - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \cdots + (-1)^n \frac{1}{2N+1}\right ) $

Um exemplo de código SERIAL (ou com apenas 1 thread, a thread main) para resolver o problema é:

```C
double factor = 1.0;
double sum = 0.0;
for (i = 0; i < n; i++, factor = -factor) {
    sum += factor/(2i+1);
}
pi = 4.0 * sum;
```

Uma ideia de paralelização, é dividir os N termos entre T threads. Vamos assumir que N é divisível por T para uma distribuição balanceada (load balance). Seja $\bar{N}=\dfrac{N}{T}$. Assim a thread(0) somará os termos na faixa $0..N-1$. A thread(1) somará os próximos $\bar{N}$ termos, na faixa $N..2N-1$. Ou seja, para a thread(q) a faixa será $q\bar{N}..(q+1)\bar{N}-1$. Se $q\bar{N}$ é par, o termo é POSITIVO, em caso contrário, é NEGATIVO.

Abaixo tem-se um exemplo de código com a variável SOMA compartilhada entre as T threads, sem controle à eventual seção crítica:

```C
void* calcPartialPI sum(void* rank) {
    long my_rank = (long) rank; //typecast
    double factor;
    long long i;
    long long my_n = n/T;
    long long my_first_i = my_n*my_rank;
    long long my_last_i = my_first_i + my_n;

    if (my_first_i % 2 == 0) /* my first i is even */
        factor = 1.0;
    else /* my first i is odd */
        factor = -1.0;

    for (i = my_first_i; i < my_last_i; i++, factor = -factor) {
            sum += factor/(2i+1);
    }

    return NULL;
} 
```

Pede-se:

a) PIparallel_1: Elaborar versão do código incluindo seção crítica controlada por MUTEX dentro do loop, tal como em: https://github.com/josenalde/flux-embedded-design/blob/main/src/pthread_count3s_mutex_1.cpp

b) PIparallel_2: Elaborar versão do código incluindo seção crítica controlada por MUTEX fora do loop, com variável privada da soma de contribuição de cada thread, tal como em: https://github.com/josenalde/flux-embedded-design/blob/main/src/pthread_count3s_mutex_3.cpp.

<img src=atividade1.png>

c) PIserial: Rodar versão do código serial, sem paralelismo, ou seja, com 1 thread

d) criar tabela comparativa das letras a) e b) para o caso de 1 thread por core e de 2 threads por core. Exemplo: se a CPU tem 4 cores e até 2 threads por core, fazer para T=4 e T=8. Se tem 08 cores, fazer para T=8 e T=16. Na tabela deixar claro qual a CPU em que está rodando a aplicação (no linux comando lscpu), detalhando número de cores/threads, níveis de cache e tamanho de cada cache (L1, L2, L3). Pode ser usado o método presente nos arquivos acima para calcular o tempo, usando clock() do C, ou o chrono do C++ ou o omp_get_wtime() do omp.h. A tabela resumo deve ter aparência como neste exemplo, comparando o caso SERIAL com os dois casos PARALELOS: formatar número de saída com 12 casas decimais, com cout.precision(12) e na hora de exibir uma variável, suponha x, fazer cout << fixed << x;

Apresentar na tabela a coluna com o PI calculado e a coluna com o tempo médio de 10 execuções para cada caso.

<img src=atividade2.png></img>

### Atividade 3.2: desenvolver código com paradigma paralelo para multiplicação matrix vetor

Seja uma matriz $A_{m \times n}$, um vetor $x_{n \times 1}$ e um vetor resultante $y_{m \times 1}$ tal que desejamos calcular o produto $Ax=y$. Esta é uma operação comum na resolução de sistemas lineares, com inúmeras aplicações em álgebra (otimização de funções com restrições etc.), transformação de imagens (rotação, aumento de escala etc.). 

Podemos compreender melhor visualizando a matriz e os vetores com seus respectivos elementos:

<img src=matriz1.png></img>

A obtenção de cada $y$ é dada pelo somatório:

$y_i=\sum_{j=0}^{n-1} a_{ij}x_j.$ Um possível pseudocódigo SERIAL pode ser:

```C
for (i = 0; i < m; i++) {
    y[i] = 0.0;
    for (j = 0; j < n; j++) {
        y[i] += A[i][j] * x[j];
    }
}
```
Uma ideia para paralelizar seria dividir as linhas (de preferência igualmente) entre as threads e cada thread calcularia um subconjunto das saídas $y$. Por exemplo, se $m=n=6$ e temos $3$ threads, a thread0 poderia resolver $y[0],y[1]$, a thread1 $y[2],y[3]$ e a thread2 $y[4],y[5]$. Ou seja, para uma thread $i$ o código seria:

```C
y[i] = 0.0;
for (j = 0; j < n; j++) y[i] += A[i][j] * x[j];
```
A forma dinâmica de atribuir os índices para cada tread $q$ seria:

start = $q \times \frac{m}{t}$

end = $(q + 1) \times \frac{m}{t} - 1$

Neste caso, não há problema com choque de atualização de variáveis globais (race condition), então o código pode ser implementado com $A, x, y, m, n$ globais e compartilhadas. Abaixo pseudocódigo para a função a ser desenvolvida com PTHREADS.

```C
void multMatrixVectorParallel(void * rank) {
    long my rank = (long) rank;
    int i, j;
    int local_m = m/thread count; // thread_count número total de threads
    int my_start = my_rank * local_m;
    int my_end = (my_rank+1) * local_m - 1;
    for (i = my_start; i <= my_end; i++) {
        y[i] = 0.0;
        for (j = 0; j < n; j++)
            y[i] += A[i][j] * x[j];
    }
    return NULL;
}
```

TAREFA: assim como no item 3.1, elaborar tabela comparativa para 1 THREAD, número de threads = número de cores e número de threads = dobro número de cores. Executar 10 ou mais vezes e colocar na tabela a média de tempo.

Testar para as seguintes matrizes com valores DOUBLE:

a) $ m=8000000,n=8 $

b) $ m = 8000, n=8000 $

c) $ m=8,n=8000000 $

DICA: preencher a matriz e o vetor $x$ com valores DOUBLE aleatórios entre 0 e 5;

